<a href="https://colab.research.google.com/github/ALA-X/Terminos-y-servicios/blob/main/DelancerLambdas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#####ORDENES DE SERVICIO
import json
import re
import time
import boto3

# ================= CONFIGURACIÓN =================
S3_OUTPUT = "s3://hba-bi/DELANCERPROD/DELAMBDA"
DATABASE = "delancerprod"
TABLE = "delancerorder"

athena = boto3.client("athena")

def run_athena_query(query):
    resp = athena.start_query_execution(
        QueryString=query,
        QueryExecutionContext={"Database": DATABASE},
        ResultConfiguration={"OutputLocation": S3_OUTPUT}
    )
    qid = resp["QueryExecutionId"]

    while True:
        st = athena.get_query_execution(QueryExecutionId=qid)
        state = st["QueryExecution"]["Status"]["State"]
        if state in ("SUCCEEDED", "FAILED", "CANCELLED"):
            break
        time.sleep(1)

    if state != "SUCCEEDED":
        reason = st["QueryExecution"]["Status"].get("StateChangeReason", "Unknown Error")
        raise Exception(f"Athena error: {state}. Reason: {reason}")

    res = athena.get_query_results(QueryExecutionId=qid, MaxResults=1000)
    rows = res["ResultSet"]["Rows"]

    if len(rows) <= 1:
        return []

    headers = [c.get("VarCharValue", "") for c in rows[0]["Data"]]
    return [
        dict(zip(headers, [c.get("VarCharValue", "N/A") for c in r["Data"]]))
        for r in rows[1:]
    ]

def lambda_handler(event, context):
    try:
        texto = event.get("inputText", "").lower().strip()
        print(f"Texto recibido: {texto}")

        where = []

        # ================= INTENCIONES =================
        is_avg = bool(re.search(r"(promedio|eficiencia|tarda|tiempo)", texto))
        is_count = bool(re.search(r"(cu[aá]ntas|total|cantidad|n[uú]mero|conteo)", texto))
        is_summary_closed = any(word in texto for word in ["por cierre", "estado de cierre"])

        # ================= FILTROS DE IDENTIDAD =================
        entidad_match = re.search(
            r"(?:cliente|tecnico|orden de|de)\s+([a-z\s]+?)(?=\s+en\s+|\s+del\s+|\s+de\s+|\s+20\d{2}|$)",
            texto
        )

        if entidad_match:
            nombre = entidad_match.group(1).strip().replace("'", "''")

            palabras_basura = [
                "de", "del", "la",
                "orden", "ordenes", "órdenes",
                "servicio", "servicios"
            ]

            if nombre and nombre not in palabras_basura:
                if "tecnico" in texto:
                    where.append(f"LOWER(technical_name) LIKE '%{nombre}%'")
                elif "cliente" in texto:
                    where.append(f"LOWER(client_fullname) LIKE '%{nombre}%'")
                else:
                    where.append(
                        f"(LOWER(client_fullname) LIKE '%{nombre}%' "
                        f"OR LOWER(technical_name) LIKE '%{nombre}%')"
                    )

        # ================= FILTRO POR CONTRATO =================
        contrato_match = re.search(r"contrato\s+(\d+)", texto)
        if contrato_match:
            contract_id = contrato_match.group(1)
            where.append(f"contract_id = '{contract_id}'")  # <-- columna corregida

        # ================= FILTRO POR AÑO (STRING) =================
        year_match = re.search(r"(20\d{2})", texto)
        if year_match:
            y = year_match.group(1)
            where.append(f"SUBSTR(order_date, 1, 4) = '{y}'")

        # ================= FILTRO POR MES =================
        meses = {
            "enero": "01", "febrero": "02", "marzo": "03", "abril": "04",
            "mayo": "05", "junio": "06", "julio": "07", "agosto": "08",
            "septiembre": "09", "octubre": "10", "noviembre": "11", "diciembre": "12"
        }

        mes_detectado = next((v for k, v in meses.items() if k in texto), None)
        if mes_detectado:
            where.append(f"SUBSTR(order_date, 6, 2) = '{mes_detectado}'")

        where_sql = "WHERE " + " AND ".join(where) if where else ""

        # ================= LÓGICA SQL =================

        if is_summary_closed:
            query = f"""
                SELECT is_closed, COUNT(*) AS total
                FROM {TABLE}
                {where_sql}
                GROUP BY is_closed
            """
            data = run_athena_query(query)

            if not data:
                body = "No se encontraron datos para el resumen."
            else:
                body = "Resumen de gestión:\n"
                for r in data:
                    estado = "Finalizadas" if str(r["is_closed"]).lower() in ["true", "1"] else "Pendientes"
                    body += f"- {estado}: {r['total']}\n"

        elif is_avg:
            query = f"""
                SELECT AVG(TRY_CAST(days_to_close AS DOUBLE)) AS promedio
                FROM {TABLE}
                {where_sql}
            """
            data = run_athena_query(query)
            val = float(data[0]["promedio"]) if data and data[0]["promedio"] != "N/A" else 0
            body = f"El tiempo promedio de atención es de {val:.2f} días."

        elif is_count:
            query = f"""
                SELECT COUNT(*) AS total
                FROM {TABLE}
                {where_sql}
            """
            data = run_athena_query(query)
            total = data[0]["total"] if data else "0"
            body = f"Se encontraron {total} órdenes para los criterios indicados."

        else:
            query = f"""
                SELECT
                    service_order_id,
                    client_fullname,
                    order_type_name,
                    order_date
                FROM {TABLE}
                {where_sql}
                ORDER BY order_date DESC
                LIMIT 10
            """
            data = run_athena_query(query)

            if not data:
                body = "No se encontraron registros."
            else:
                body = "Listado de órdenes:\n"
                body += "\n".join([
                    f"- ID {r['service_order_id']}: {r['order_type_name']} ({r['order_date']})"
                    for r in data
                ])

        return {
            "messageVersion": "1.0",
            "response": {
                "actionGroup": "action_group_services",
                "function": "services_function",
                "functionResponse": {
                    "responseBody": {
                        "TEXT": {
                            "body": body
                        }
                    }
                }
            }
        }

    except Exception as e:
        return {
            "messageVersion": "1.0",
            "response": {
                "actionGroup": "action_group_services",
                "function": "services_function",
                "functionResponse": {
                    "responseBody": {
                        "TEXT": {
                            "body": f"Error: {str(e)}"
                        }
                    }
                }
            }
        }


In [1]:
###CONTRATO
import json
import re
import time
import boto3

# ================= CONFIGURACIÓN =================
S3_OUTPUT = "s3://hba-bi/DELANCERPROD/DELAMBDA"
DATABASE = "delancerprod"
TABLE = "delancercontract"

athena = boto3.client("athena")
LAST_CONTRACT = None

# ================= HELPERS =================
def run_athena_query(query):
    resp = athena.start_query_execution(
        QueryString=query,
        QueryExecutionContext={"Database": DATABASE},
        ResultConfiguration={"OutputLocation": S3_OUTPUT}
    )
    qid = resp["QueryExecutionId"]

    while True:
        st = athena.get_query_execution(QueryExecutionId=qid)
        state = st["QueryExecution"]["Status"]["State"]
        if state in ("SUCCEEDED", "FAILED", "CANCELLED"):
            break
        time.sleep(1)

    if state != "SUCCEEDED":
        reason = st["QueryExecution"]["Status"].get("StateChangeReason", "Unknown")
        raise Exception(f"Athena query failed: {state}. Reason: {reason}")

    res = athena.get_query_results(QueryExecutionId=qid, MaxResults=1000)
    rows = res["ResultSet"]["Rows"]

    headers = [c.get("VarCharValue", "") for c in rows[0]["Data"]]
    data = []
    for r in rows[1:]:
        vals = [c.get("VarCharValue", "N/A") for c in r["Data"]]
        data.append(dict(zip(headers, vals)))

    return data

# ================= FUNCIONES AUXILIARES =================
def build_full_summary(contract_data, payments_data, equipment_data, services_data):
    """Construye el resumen completo con todas las secciones"""
    r = contract_data[0] if contract_data else {}
    contract_section = {
        "fullname": r.get("fullname", "N/A"),
        "code": r.get("contract_id", "N/A"),
        "locality": r.get("locality_name", "N/A"),
        "status": r.get("status_name", "N/A"),
        "days_since_install": r.get("days_since_install", "N/A"),
        "address": r.get("sector_name", "N/A"),
        "last_payment_date": r.get("last_payment_date", "N/A"),
        "last_cut_date": r.get("last_cut_date", "N/A")
    }

    payments_section = {
        "balance_actual": payments_data[0].get("current_balance_amount", "N/A") if payments_data else "N/A",
        "estado_balance": "Al día" if payments_data and float(payments_data[0].get("current_balance_amount",0)) >= 0 else "Vencido",
        "dias_atraso": payments_data[0].get("dias_atraso", "N/A") if payments_data else "N/A"
    }

    equipment_section = {
        "tipo_equipo": equipment_data[0].get("tipo_equipo", "N/A") if equipment_data else "N/A",
        "plan": equipment_data[0].get("plan", "N/A") if equipment_data else "N/A",
        "estado": equipment_data[0].get("estado", "N/A") if equipment_data else "N/A"
    }

    services_section = {
        "ultimas_ordenes": services_data if services_data else "N/A"
    }

    summary = f"""
=== Información del Contrato ===
{json.dumps(contract_section, indent=2)}

=== Información Financiera ===
{json.dumps(payments_section, indent=2)}

=== Equipos y Planes Contratados ===
{json.dumps(equipment_section, indent=2)}

=== Órdenes de Servicio ===
{json.dumps(services_section, indent=2)}
"""
    return summary

def run_contract_query(contract_id):
    query = f"""
        SELECT contract_id, fullname, status_name, locality_name, sector_name,
               last_cut_date, last_payment_date, last_suspend_date, days_since_install
        FROM {TABLE}
        WHERE CAST(contract_id AS VARCHAR) = '{contract_id}'
        LIMIT 1
    """
    return run_athena_query(query)

# ================= LAMBDA =================
def lambda_handler(event, context):
    global LAST_CONTRACT

    try:
        texto = event.get("inputText", "").lower().strip()
        print("Texto recibido:", texto)

        where = []
        order_field = "contract_id"
        date_field = "contract_date"  # Default para orden

        # -------- Detectar Resumen Completo --------
        is_full_summary = bool(re.search(r"resumen completo|visión 360|ficha del cliente|estado general", texto))

        # -------- Intenciones --------
        wants_list = "listado" in texto or "detalle" in texto
        is_count = not wants_list and bool(re.search(r"(total|cu[aá]ntos|n[uú]mero)", texto))

        # -------- Filtros: Año --------
        year_match = re.search(r"(20\d{2})", texto)
        year = year_match.group(1) if year_match else None

        # -------- Filtros: Localidad --------
        loc_match = re.search(r"\b(sosua|cabarete|puerto plata|santiago|rio san juan)\b", texto)
        if loc_match:
            where.append(f"LOWER(locality_name) = '{loc_match.group(1)}'")

        # -------- Filtros: Contrato ID --------
        cid = re.search(r"\bcontrato\s+(\d+)\b", texto)
        if cid:
            LAST_CONTRACT = cid.group(1)
            where.append(f"CAST(contract_id AS VARCHAR) = '{LAST_CONTRACT}'")

        # -------- Filtros: Nombre --------
        name_match = re.search(r"(?:cliente|contrato)(?:\s+de\s+la|\s+de\s+|\s+del\s+|\s+)([a-z\s]+?)(?=\s+en\s+|\s+del\s+|\s+\d{4}|\b(sosua|cabarete|activo|cortado|pendiente|suspendido)\b|$)", texto)
        if name_match and not cid:
            n = name_match.group(1).strip().replace("'", "''")
            if n not in ["de", "del", "la"]:
                where.append(f"LOWER(fullname) LIKE LOWER('%{n}%')")

        # -------- Filtros: Estados Dinámicos --------
        estados_posibles = ["activo", "cortado", "suspendido", "pendiente", "retirado", "anulado"]
        for est in estados_posibles:
            if est in texto:
                where.append(f"LOWER(status_name) = '{est}'")
                if est == "activo": date_field = "last_payment_date"
                if est == "cortado": date_field = "last_cut_date"
                if est == "suspendido": date_field = "last_suspend_date"
                break

        # -------- Filtro de Año Aplicado --------
        if year:
            if is_count:
                # Para conteos, usamos last_payment_date en lugar de contract_date
                f_year = "last_payment_date"
            else:
                f_year = date_field
            where.append(f"({f_year} LIKE '{year}%' OR {f_year} IS NULL)")

        # -------- SQL Building --------
        order_sql = f"CASE WHEN {date_field} LIKE '____-__-__' THEN {date_field} ELSE '0000-00-00' END"
        where_sql = "WHERE " + " AND ".join(where) if where else ""

        # -------- Resumen Completo --------
        if is_full_summary and LAST_CONTRACT:
            contract_data = run_contract_query(LAST_CONTRACT)

            # Llamadas a otros action groups (simuladas aquí)
            payments_data = []  # Aquí deberías llamar a payments_function con contract_id
            equipment_data = [] # Aquí deberías llamar a equipment_function con contract_id
            services_data = []  # Aquí deberías llamar a services_function con contract_id

            body = build_full_summary(contract_data, payments_data, equipment_data, services_data)
        else:
            if is_count:
                query = f"SELECT COUNT(*) AS total FROM {TABLE} {where_sql}"
                rows = run_athena_query(query)
                total = rows[0]["total"] if rows else "0"

                criterios = []
                if loc_match: criterios.append(f"en {loc_match.group(1).capitalize()}")
                if year: criterios.append(f"del año {year}")
                criterios_str = " ".join(criterios)

                body = f"Se encontraron {total} contratos {criterios_str}."
            else:
                query = f"""
                    SELECT contract_id, fullname, status_name, locality_name, sector_name,
                           last_cut_date, last_payment_date, last_suspend_date
                    FROM {TABLE} {where_sql}
                    ORDER BY {order_sql} DESC LIMIT 10
                """
                rows = run_athena_query(query)

                if not rows:
                    body = "No se encontraron contratos con esos criterios."
                elif len(rows) == 1 and not wants_list:
                    r = rows[0]
                    body = f"El contrato {r['contract_id']} de {r['fullname']} está actualmente {r['status_name']} en {r['locality_name']}."
                else:
                    body = "Resultado de contratos:\n" + "\n".join(
                        "- " + " | ".join(f"{k}: {v}" for k, v in r.items()) for r in rows
                    )

        return {
            "messageVersion": "1.0",
            "response": {
                "actionGroup": "action_group_contract",
                "function": "delancercontract",
                "functionResponse": {
                    "responseBody": {"TEXT": {"body": body}}
                }
            }
        }

    except Exception as e:
        return {
            "messageVersion": "1.0",
            "response": {
                "actionGroup": "action_group_contract",
                "function": "delancercontract",
                "functionResponse": {
                    "responseBody": {"TEXT": {"body": f"Error: {str(e)}"}}
                }
            }
        }


In [ ]:
####PLANES
import json
import re
import time
import boto3

# ================= CONFIGURACIÓN =================
S3_OUTPUT = "s3://hba-bi/DELANCERPROD/DELAMBDA"
DATABASE = "delancerprod"
TABLE = "delancerequip"

athena = boto3.client("athena")

# ================= HELPERS =================

def run_athena_query(query):
    response = athena.start_query_execution(
        QueryString=query,
        QueryExecutionContext={"Database": DATABASE},
        ResultConfiguration={"OutputLocation": S3_OUTPUT},
    )

    execution_id = response["QueryExecutionId"]

    while True:
        result = athena.get_query_execution(QueryExecutionId=execution_id)
        state = result["QueryExecution"]["Status"]["State"]
        if state in ["SUCCEEDED", "FAILED", "CANCELLED"]:
            break
        time.sleep(0.5)

    if state != "SUCCEEDED":
        reason = result["QueryExecution"]["Status"].get("StateChangeReason", "Unknown")
        raise Exception(f"Error ejecutando la consulta Athena: {reason}")

    result = athena.get_query_results(QueryExecutionId=execution_id)
    rows = result["ResultSet"]["Rows"]

    if len(rows) <= 1:
        return []

    headers = [c.get("VarCharValue", "") for c in rows[0]["Data"]]
    data = []

    for row in rows[1:]:
        values = [c.get("VarCharValue", "") for c in row["Data"]]
        data.append(dict(zip(headers, values)))

    return data

# ================= NLP =================

def detect_intent(text):
    text = text.lower()

    if re.search(r"planes.*contrato", text):
        return "plans_by_contract"

    if re.search(r"cuantos equipos.*contrato|equipos.*contrato", text):
        return "equipment_by_type_and_contract"

    if re.search(r"equipos activos|equipos pendientes|equipos suspendidos|equipos cortados", text):
        return "equipment_by_status"

    # Se agregan variantes para mejorar la detección
    if re.search(r"que equipos tiene|equipos del cliente|equipos de|equipos por", text):
        return "equipment_by_client"

    if re.search(r"planes del cliente|planes tiene el cliente", text):
        return "plans_by_client"

    if re.search(r"planes más vendidos|plan más vendido", text):
        return "top_plans"

    if re.search(r"equipos por tipo|internet|tv", text):
        return "equipment_by_type"

    return "unknown"

def extract_client_name(text):
    # CORRECCIÓN:
    # 1. Busca disparadores: cliente, tiene, de, del, por
    # 2. Ignora opcionalmente artículos: la, el, los, las
    # 3. Ignora opcionalmente la palabra repetida: cliente
    # 4. Captura el nombre real (letras y espacios)
    match = re.search(
        r"(?:cliente|tiene|de|del|por)\s+(?:la\s+|el\s+|los\s+|las\s+)?(?:cliente\s+)?([a-zA-ZñÑÁÉÍÓÚáéíóú\s]+)",
        text,
        re.IGNORECASE
    )

    if match:
        name = match.group(1).strip()
        # Elimina posibles palabras de cierre o cortesía que se cuelen en la captura
        name = re.split(r"\s+(?:por\s+favor|gracias|consulta|de)\b", name, flags=re.IGNORECASE)[0]
        return name.strip()
    return None

def extract_contract_id(text):
    match = re.search(r"contrato\s*(?:número|nro|no\.?)?\s*:?\s*(\d+)", text, re.IGNORECASE)
    return match.group(1) if match else None

def extract_status(text):
    estados = ["activo", "pendiente", "suspendido", "cortado", "cancelado"]
    for e in estados:
        if e in text.lower():
            return e.upper()
    return None

# ================= HANDLER =================

def lambda_handler(event, context):
    try:
        user_text = event.get("inputText") or event.get("text") or ""
        print("Texto recibido:", user_text)

        intent = detect_intent(user_text)
        status = extract_status(user_text)
        contract_id = extract_contract_id(user_text)

        # ================= QUERIES =================

        if intent == "equipment_by_type_and_contract":
            if not contract_id:
                body = "Indica el número de contrato."
            else:
                query = f"""
                SELECT equipment_type, COUNT(*) AS total
                FROM {TABLE}
                WHERE CAST(contract_id AS VARCHAR) = '{contract_id}'
                GROUP BY equipment_type
                """
                result = run_athena_query(query)
                body = (
                    f"Equipos del contrato {contract_id} por tipo:\n" +
                    "\n".join([f"- {r['equipment_type']}: {r['total']}" for r in result])
                ) if result else "El contrato no tiene equipos registrados."

        elif intent == "equipment_by_status":
            where = f"WHERE equipment_status = '{status}'" if status else ""
            query = f"SELECT equipment_status, COUNT(*) AS total FROM {TABLE} {where} GROUP BY equipment_status"
            result = run_athena_query(query)
            body = "Equipos por estado:\n" + "\n".join([f"{r['equipment_status']}: {r['total']}" for r in result])

        elif intent == "equipment_by_client":
            client = extract_client_name(user_text)
            if not client:
                body = "Indica el nombre del cliente."
            else:
                query = f"""
                SELECT serial, equipment_type, plan_name, equipment_status
                FROM {TABLE}
                WHERE LOWER(contract_fullname) LIKE LOWER('%{client}%')
                """
                result = run_athena_query(query)
                body = (
                    f"Equipos del cliente {client.upper()}:\n" +
                    "\n".join([f"{r['equipment_type']} | {r['plan_name']} ({r['equipment_status']})" for r in result])
                ) if result else f"No se encontraron equipos para el cliente: {client.upper()}"

        elif intent == "plans_by_client":
            client = extract_client_name(user_text)
            if not client:
                body = "Indica el nombre del cliente."
            else:
                query = f"""
                SELECT plan_name, COUNT(*) AS total
                FROM {TABLE}
                WHERE LOWER(contract_fullname) LIKE LOWER('%{client}%')
                GROUP BY plan_name
                """
                result = run_athena_query(query)
                body = (
                    f"Planes del cliente {client.upper()}:\n" +
                    "\n".join([f"{r['plan_name']}: {r['total']}" for r in result])
                ) if result else "El cliente no tiene planes registrados."

        elif intent == "plans_by_contract":
            if not contract_id:
                body = "Indica el número de contrato."
            else:
                query = f"""
                SELECT plan_name, equipment_type, equipment_status, serial
                FROM {TABLE}
                WHERE CAST(contract_id AS VARCHAR) = '{contract_id}'
                """
                result = run_athena_query(query)
                if not result:
                    body = f"El contrato {contract_id} no tiene planes registrados."
                else:
                    body = f"Planes y servicios del contrato {contract_id}:\n"
                    for r in result:
                        emoji = "📺" if "tv" in r['equipment_type'].lower() else "🌐"
                        body += f"{emoji} {r['equipment_type']}: {r['plan_name']} | Estado: {r['equipment_status']} (S/N: {r['serial']})\n"

        elif intent == "top_plans":
            query = f"SELECT plan_name, COUNT(*) AS total FROM {TABLE} GROUP BY plan_name ORDER BY total DESC LIMIT 5"
            result = run_athena_query(query)
            body = "Planes más vendidos:\n" + "\n".join([f"{r['plan_name']}: {r['total']}" for r in result])

        elif intent == "equipment_by_type":
            query = f"SELECT equipment_type, COUNT(*) AS total FROM {TABLE} GROUP BY equipment_type"
            result = run_athena_query(query)
            body = "Equipos por tipo:\n" + "\n".join([f"- {r['equipment_type']}: {r['total']}" for r in result])

        else:
            body = "No pude identificar la consulta. Puedo ayudarte con equipos o planes."

        return {
            "messageVersion": "1.0",
            "response": {
                "actionGroup": "action_group_equipment",
                "function": "equipment_function",
                "functionResponse": {"responseBody": {"TEXT": {"body": body}}}
            }
        }

    except Exception as e:
        return {
            "messageVersion": "1.0",
            "response": {
                "actionGroup": "action_group_equipment",
                "function": "equipment_function",
                "functionResponse": {"responseBody": {"TEXT": {"body": f"Error: {str(e)} "}}}
            }
        }

In [ ]:
##PAGOS
import json
import re
import time
import boto3

# ================= CONFIGURACIÓN =================
S3_OUTPUT = "s3://hba-bi/DELANCERPROD/DELAMBDA"
DATABASE = "delancerprod"
TABLE = "delancerpay"
athena = boto3.client("athena")
LAST_PAYER = None

# ================= HELPERS =================
def run_athena_query(query):
    resp = athena.start_query_execution(
        QueryString=query,
        QueryExecutionContext={"Database": DATABASE},
        ResultConfiguration={"OutputLocation": S3_OUTPUT}
    )
    qid = resp["QueryExecutionId"]

    while True:
        st = athena.get_query_execution(QueryExecutionId=qid)
        state = st["QueryExecution"]["Status"]["State"]
        if state in ("SUCCEEDED", "FAILED", "CANCELLED"):
            break
        time.sleep(1)

    if state != "SUCCEEDED":
        reason = st["QueryExecution"]["Status"].get("StateChangeReason", "Unknown")
        raise Exception(f"Athena query failed: {state}. Reason: {reason}")

    res = athena.get_query_results(QueryExecutionId=qid, MaxResults=1000)
    rows = res["ResultSet"]["Rows"]

    headers = [c.get("VarCharValue", "") for c in rows[0]["Data"]]
    data = []
    for r in rows[1:]:
        vals = [c.get("VarCharValue", "0") for c in r["Data"]]
        data.append(dict(zip(headers, vals)))
    return data

# ================= LAMBDA =================
def lambda_handler(event, context):
    global LAST_PAYER

    try:
        texto = event.get("inputText", "").lower().strip()
        print("Texto recibido:", texto)

        where = []

        # -------- Intenciones --------
        is_income_request = bool(re.search(r"(ingreso|recauda|cobr[oó]|sumar|suma|monto|dinero)", texto))
        wants_list = any(word in texto for word in ["listado", "detalle", "resumen", "desglose"])
        is_balance_request = any(word in texto for word in ["balance", "deuda", "saldo"])
        by_locality = any(word in texto for word in ["por localidad", "por ciudad", "por zona", "por regiones"])

        is_count = (
            not is_income_request and not wants_list and not is_balance_request and
            bool(re.search(r"(total|cu[aá]ntos|n[uú]mero|cantidad)", texto))
        )

        # -------- Filtros Temporales --------
        year_match = re.search(r"(20\d{2})", texto)
        year = year_match.group(1) if year_match else None
        if year:
            where.append(f"substr(payment_date,1,4) = '{year}'")

        meses = {
            "enero": "01", "febrero": "02", "marzo": "03", "abril": "04", "mayo": "05", "junio": "06",
            "julio": "07", "agosto": "08", "septiembre": "09", "octubre": "10", "noviembre": "11", "diciembre": "12"
        }
        mes_detectado = next((v for k, v in meses.items() if k in texto), None)
        if mes_detectado:
            where.append(f"substr(payment_date,6,2) = '{mes_detectado}'")

        # -------- Filtros de Identidad --------
        payer = re.search(r"(cliente|usuario|payer)\s+([a-z\s]+?)(?=\s+en\s+|\s+del\s+|\s+de\s+|\s+\d{4}|$)", texto)
        if payer:
            LAST_PAYER = payer.group(2).strip().replace("'", "''")
            where.append(f"LOWER(payer_name) LIKE LOWER('%{LAST_PAYER}%')")

        doc = re.search(r"\b\d{8,13}\b", texto)
        if doc:
            where.append(f"payer_identification = '{doc.group(0)}'")

        # -------- FILTRO POR CONTRACT_KEY --------
        contract_key_match = re.search(r"contrato\s*(?:número|nro|no\.?|:)?\s*(\d+)", texto, re.IGNORECASE)
        if contract_key_match:
            code = contract_key_match.group(1)
            where.append(f"CAST(contract_key AS VARCHAR) = '{code}'")

        where_sql = "WHERE " + " AND ".join(where) if where else ""

        # ================= LÓGICA DE SQL =================
        is_max_payment_date = bool(re.search(r"(fecha m[aá]xima de pago|fecha tope de pago|ultima fecha de pago|última fecha de pago)", texto))

        if is_max_payment_date:
            query = f"SELECT MAX(CAST(payment_date AS DATE)) AS fecha_maxima_pago FROM {TABLE} {where_sql}"
            rows = run_athena_query(query)
            res_fecha = rows[0]["fecha_maxima_pago"] if rows else None
            body = res_fecha if res_fecha and res_fecha != "0" else "No se encontró fecha."

        elif is_income_request:
            if by_locality:
                # --- NUEVO: resumen por todas las localidades ---
                query = f"""
                    SELECT locality_name, SUM(CAST(total_amount AS DOUBLE)) AS monto
                    FROM {TABLE} {where_sql}
                    GROUP BY locality_name
                    ORDER BY monto DESC
                """
                rows = run_athena_query(query)
                periodo = f"en {year}" if year else "total"
                if rows:
                    body = f"Ingresos por localidad ({periodo}):\n" + "\n".join(
                        [f"- {r['locality_name']}: ${float(r['monto']):,.2f}" for r in rows]
                    )
                else:
                    body = f"No se encontraron registros de ingresos {periodo} por localidad."

            elif wants_list and year and not mes_detectado:
                query = f"""
                    SELECT substr(payment_date, 1, 7) AS mes, SUM(CAST(total_amount AS DOUBLE)) AS monto
                    FROM {TABLE} {where_sql}
                    GROUP BY substr(payment_date, 1, 7)
                    ORDER BY mes ASC
                """
                rows = run_athena_query(query)
                body = f"Resumen de recaudación por mes ({year}):\n" + "\n".join(
                    [f"- {r['mes']}: ${float(r['monto']):,.2f}" for r in rows]
                )
            else:
                query = f"SELECT SUM(CAST(total_amount AS DOUBLE)) AS total FROM {TABLE} {where_sql}"
                rows = run_athena_query(query)
                valor = float(rows[0]["total"]) if rows and rows[0]["total"] != "0" else 0.0
                body = f"El valor recaudado según los filtros es: ${valor:,.2f}"

        elif is_count:
            query = f"SELECT COUNT(*) AS total_pagos FROM {TABLE} {where_sql}"
            rows = run_athena_query(query)
            body = f"{rows[0]['total_pagos']} pagos encontrados."

        else:
            query = f"""
                SELECT
                    payment_key, payment_id, receipt_number, payer_name,
                    payer_identification, contract_key, contract_code, payment_date,
                    total_amount, current_balance_amount, payment_method,
                    status_contract_name, locality_name
                FROM {TABLE}
                {where_sql}
                ORDER BY CAST(payment_date AS DATE) DESC
                LIMIT 1
            """
            rows = run_athena_query(query)

            if not rows:
                body = "No se encontraron registros con los criterios indicados."
            elif is_balance_request and not wants_list:
                r = rows[0]
                body = f"El cliente {r['payer_name']} registra un balance de ${float(r['current_balance_amount']):,.2f} en función a la última fecha {r['payment_date']}."
            else:
                body = "Resultado de pagos:\n" + "\n".join(
                    "- " + " | ".join(f"{k}: {v}" for k, v in r.items())
                    for r in rows
                )

        return {
            "messageVersion": "1.0",
            "response": {
                "actionGroup": "action_group_payments",
                "function": "payments_function",
                "functionResponse": {
                    "responseBody": {
                        "TEXT": {"body": body}
                    }
                }
            }
        }

    except Exception as e:
        return {
            "messageVersion": "1.0",
            "response": {
                "actionGroup": "action_group_payments",
                "function": "payments_function",
                "functionResponse": {
                    "responseBody": {
                        "TEXT": {"body": f"Error interno: {str(e)}"}
                    }
                }
            }
        }
